In [40]:
import tensorflow as tf
%run pose_labels_dict.py

{1: 'Hello', 2: 'Thank You', 0: 'Peace', 3: 'Good Luck', 4: 'Random', 5: 'Break', 6: 'Nice', 7: 'Fuck You'}


In [41]:
model = tf.keras.models.load_model("pose_model_2")

In [42]:
import numpy as np
demo_data = np.random.rand(1, 15, 42)
demo_data

array([[[0.11898911, 0.22200714, 0.18821187, 0.95068754, 0.0512857 ,
         0.96653321, 0.54197905, 0.08241895, 0.39347787, 0.15478781,
         0.17444318, 0.82783901, 0.86542114, 0.7902957 , 0.03791342,
         0.89804545, 0.44688502, 0.52604028, 0.35053468, 0.71525558,
         0.45193669, 0.39825803, 0.26968191, 0.88992052, 0.80501459,
         0.21018876, 0.13906451, 0.5786002 , 0.11173103, 0.53420634,
         0.58267633, 0.25535786, 0.85045485, 0.49543646, 0.40804615,
         0.2308171 , 0.29440881, 0.88128042, 0.75424339, 0.85353472,
         0.01625898, 0.74820825],
        [0.43469253, 0.61589037, 0.25745133, 0.29841269, 0.01892674,
         0.44087388, 0.69558097, 0.10150361, 0.23187331, 0.08765817,
         0.39655427, 0.30272539, 0.17222328, 0.66090294, 0.16289338,
         0.35071243, 0.77378055, 0.34900692, 0.04783599, 0.91595284,
         0.74846008, 0.3342537 , 0.10860261, 0.57513915, 0.10450483,
         0.70602823, 0.04499486, 0.16702178, 0.9619017 , 0.88811226,


In [43]:
# Normalize the demo data to have values between 0 and 1
demo_data_normalized = demo_data / demo_data.max()

# Make predictions
predictions = model.predict(demo_data_normalized)

# Get the predicted class index (assuming it's a classification task)
predicted_class = np.argmax(predictions)

print(f"Predicted class index: {predicted_class}")

1/1 [==============================] - 1s 548ms/step
Predicted class index: 3


In [44]:
import cv2
import mediapipe as mp 

In [45]:
hand_detector = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils
face_detector = mp.solutions.face_mesh.FaceMesh()


In [46]:
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 0, 255)  # White color in BGR
text = "predicting..."
font_scale = 1
thickness = 2
text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]

In [47]:
pose_data = PoseData()
labels = pose_data.fetch_labels()

In [60]:

cap = cv2.VideoCapture(0)
video = []
text = "Predicting..."
while True :

    _,frame = cap.read()
    frame = cv2.flip(frame , 1)
    rgb_frame = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    hand_output = hand_detector.process(rgb_frame)
    hands = hand_output.multi_hand_landmarks
    face_output = face_detector.process(rgb_frame)
    faces = face_output.multi_face_landmarks
    fheight , fwidth , _ = frame.shape

    if faces:
        for face in faces:
            drawing_utils.draw_landmarks(frame , face , mp.solutions.face_mesh.FACEMESH_TESSELATION , None , mp.solutions.drawing_styles.DrawingSpec(color=(0,0,255),thickness=1,circle_radius=1))
            drawing_utils.draw_landmarks(frame , face , mp.solutions.face_mesh.FACEMESH_CONTOURS , None , mp.solutions.drawing_styles.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=1))
            # drawing_utils.draw_landmarks(frame , face , mp.solutions.face_mesh.FACEMESH_IRISES , None , mp.solutions.drawing_styles.get_default_face_mesh_iris_connections_style())
    
    if hands :
        hand = hands[0]
        drawing_utils.draw_landmarks(frame , hand , mp.solutions.hands.HAND_CONNECTIONS )
        videofeatures = []
        landmarks = hand.landmark
        
        for id , landmark in enumerate(landmarks):
            x = int(landmark.x*fwidth)
            y = int(landmark.y*fheight)
            cv2.circle(frame , (x,y) , 5 , (255,0,0) , 3)
            videofeatures.append(landmark.x)
            videofeatures.append(landmark.y)
        
        video.append(videofeatures)
        print(len(video))
        
        if(len(video) == 15):
            input_data = np.array([video])
            # print(input_data)
            # input_data = input_data.reshape((input_data.shape[0], 1, input_data.shape[1]))
            prediction = model.predict(input_data)
            # Get the predicted class index (assuming it's a classification task)
            predicted_class = np.argmax(prediction)

            print(f"Predicted class index: {predicted_class}")
            text = labels[predicted_class]
            video.remove(video[0])
    else:
        text = "Predicting..."
            
            
    position = (fwidth - text_size[0] - 10, 30)
    cv2.putText(frame, str(text), position, font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.imshow("myhands",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        
        break
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1/1 [==============================] - 0s 21ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 18ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 23ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 20ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 18ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 [==============================] - 0s 19ms/step
Predicted class index: 0
15
1/1 